<a href="https://colab.research.google.com/github/efemehmetkarabulut/AYRIK-SISTEMLER-ILERI-OLASILIK/blob/main/bert_classification_score_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

def setup_tensorflow_keras():
    # Eski sürümleri kaldır
    os.system("pip uninstall -y tensorflow keras tensorboard tf-keras")

    # Uyumlu TensorFlow ve Keras sürümlerini yükle
    os.system("pip install tensorflow==2.17.1 keras==3.5.0")

    # Bağımlı paketleri güncelle
    os.system("pip install --upgrade tensorflow-io tensorflow-addons")

    # Paket uyumluluğunu kontrol et ve kararlı sürümleri sabitle
    os.system("pip freeze > requirements.txt")
    os.system("pip install -r requirements.txt")

    print("TensorFlow ve Keras kurulumu tamamlandı. Lütfen kodunuzu çalıştırmayı deneyin.")

if __name__ == "__main__":
    setup_tensorflow_keras()


TensorFlow ve Keras kurulumu tamamlandı. Lütfen kodunuzu çalıştırmayı deneyin.


In [8]:
pip install tensorflow keras


  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached keras-3.8.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.3 MB)
Using cached keras-3.8.0-py3-none-any.whl (1.3 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)


In [10]:
!python -c "import tensorflow as tf; print(tf.__version__)"

2025-01-10 20:41:17.195047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736541677.218665    8409 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736541677.225656    8409 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 20:41:17.249859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2.18.0


In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [15]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [16]:
import tensorflow as tf
print("TensorFlow Sürümü:", tf.__version__)
print("GPU Kullanılabilir mi?", tf.config.list_physical_devices('GPU'))


TensorFlow Sürümü: 2.18.0
GPU Kullanılabilir mi? []


In [19]:
import tensorflow as tf
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
from google.colab import files

# Veri setini yükleme
def download_dataset():
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    data = pd.read_excel(file_path)
    return data

# Veri temizleme
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

# Etiketleri dönüştürme
def preprocess_data(data):
    label_encoder = LabelEncoder()
    data['category'] = label_encoder.fit_transform(data['category'])
    return data, label_encoder

# Veri setini hazırlama
def prepare_dataset(data, tokenizer):
    inputs = tokenizer(
        data['text'].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )
    input_ids = tf.convert_to_tensor(inputs["input_ids"], dtype=tf.int32)
    attention_mask = tf.convert_to_tensor(inputs["attention_mask"], dtype=tf.int32)
    labels = tf.convert_to_tensor(data['category'].values, dtype=tf.int32)
    return input_ids, attention_mask, labels

# Model oluşturma
def create_bert_model():
    bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
    return bert_model

# Modeli eğitme
def train_bert_model(model, input_ids, attention_mask, labels):
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Modeli derle
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

    # Eğitimi başlat
    model.fit(
        x={"input_ids": input_ids, "attention_mask": attention_mask},
        y=labels,
        validation_split=0.2,
        epochs=3,
        batch_size=16
    )
    return model


# Sonuçları raporlama
def report_results(model, input_ids, attention_mask, labels, label_encoder):
    predictions = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})
    preds = tf.argmax(predictions.logits, axis=-1).numpy()
    report = classification_report(labels, preds, target_names=label_encoder.classes_)
    print(report)

# Ana fonksiyon
def main():
    # Veri setini yükle
    data = download_dataset()
    data['text'] = data['text'].apply(clean_text)
    data, label_encoder = preprocess_data(data)

    # Tokenizer ve veri hazırlama
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    input_ids, attention_mask, labels = prepare_dataset(data, tokenizer)

    # Model oluştur ve eğit
    bert_model = create_bert_model()
    trained_model = train_bert_model(bert_model, input_ids, attention_mask, labels)

    # Sonuçları raporla
    report_results(trained_model, input_ids, attention_mask, labels, label_encoder)

if __name__ == "__main__":
    main()


Saving bbc_data_1_2 (1).xlsx to bbc_data_1_2 (1).xlsx


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
112/112 [==============================] - 2966s 26s/step - loss: 0.5406 - accuracy: 0.8747 - val_loss: 3.6674 - val_accuracy: 0.0944
Epoch 2/3
112/112 [==============================] - 2906s 26s/step - loss: 0.0605 - accuracy: 0.9888 - val_loss: 4.1660 - val_accuracy: 0.0989
Epoch 3/3
70/70 [==============================] - 1056s 15s/step
               precision    recall  f1-score   support

     business       0.64      1.00      0.78       510
entertainment       0.79      1.00      0.88       386
     politics       0.98      0.99      0.99       417
        sport       0.99      1.00      1.00       511
         tech       0.00      0.00      0.00       401

     accuracy                           0.82      2225
    macro avg       0.68      0.80      0.73      2225
 weighted avg       0.69      0.82      0.75      2225



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
pip install tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00


In [6]:
!which python  # Linux/Mac
!where python  # Windows

/usr/local/bin/python
/bin/bash: line 1: where: command not found
